In [20]:
import numpy as np
from sklearn.metrics import pairwise_distances_chunked


In [21]:
root_size = 5

x = np.arange(28).reshape((4,7))
diag_mask = np.identity(root_size, dtype=bool)

In [22]:
# x[diag_mask] = 0

In [23]:
x

array([[ 0,  1,  2,  3,  4,  5,  6],
       [ 7,  8,  9, 10, 11, 12, 13],
       [14, 15, 16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25, 26, 27]])

In [24]:
def triu_with_offset(x: np.array, k: int = 0):

    row_count = 0
    for row in range(x.shape[0]):
        for col in range(k + row_count, x.shape[1]):
            print(x[row,col])

        row_count += 1

In [25]:
rows, cols = np.triu_indices_from(x, k=0)
start = 0

cols+=start
mask = cols < x.shape[1]
cols = cols[mask]
rows = rows[mask]
x[rows,cols]

array([ 0,  1,  2,  3,  4,  5,  6,  8,  9, 10, 11, 12, 13, 16, 17, 18, 19,
       20, 24, 25, 26, 27])

In [ ]:
def get_mean_euclidean_chunked(X: np.ndarray) -> float:
    """
    Compute the mean pairwise Euclidean distance using chunking.
    """
    total_sum = 0.0
    total_size = 0

    def reduce_func(D_chunk, start):
        nonlocal total_sum, total_size
        n_samples = D_chunk.shape[1]

        
        row_indices = np.arange(start, start + D_chunk.shape[0])

        for local_row, global_row in enumerate(row_indices):
            
            valid_cols = np.arange(global_row + 1, n_samples)
            dists = D_chunk[local_row, valid_cols]

            total_sum += dists.sum()
            total_size += dists.size

        return None  

    
    list(pairwise_distances_chunked(X, reduce_func=reduce_func))

    return total_sum / total_size


    

In [27]:
n_observations_X = 5000_0
n_fields = 2

arr = np.random.randn(n_observations_X, n_fields)


In [36]:
get_mean_euclidean_chunked(arr)


np.float64(1.7668629855138127)

In [31]:
# take a vertical slice
slices = 3
slices_rows = 3

for slice in range(slices):
    base = slice * slices_rows 
    top = base + slices_rows 

    lhs_box_base = (slice - 1) * slices_rows
    lhs_box_top = lhs_box_base + slices_rows 

    chunk_D = x[base : top]
    print(chunk_D)
    # print(chunk_D[bl_mask])
    # print(chunk_D.shape[0])

    lhs_box = chunk_D.T[0 : lhs_box_top].T
    rhs_box = chunk_D.T[lhs_box_top:].T
    bl_mask = ~np.triu(np.ones_like(rhs_box, dtype=bool))

    tri = rhs_box[bl_mask]
    print(lhs_box)
    print(tri)
    print("", end="\n")

[[ 0  1  2  3  4  5  6]
 [ 7  8  9 10 11 12 13]
 [14 15 16 17 18 19 20]]
[]
[ 7 14 15]

[[21 22 23 24 25 26 27]]
[[21 22 23]]
[]

[]
[]
[]

